In [16]:
import matplotlib.pyplot as plt

In [1]:
#Need to run all Data Prep Notebook cells
%store -r data

In [2]:
data.head()

,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real
ID,,,,,,,,,,,,,
1000002330,108,12,5,2015-10-09,1000.0,2015-08-11 12:12:28,0.0,False,0,9,0.0,0.0,1533.95
1000003930,93,6,13,2017-11-01,30000.0,2017-09-02 04:43:57,2421.0,False,15,22,100.0,2421.0,30000.00
1000004038,93,6,13,2013-02-26,45000.0,2013-01-12 00:20:50,220.0,False,3,22,220.0,220.0,45000.00
1000007540,90,10,13,2012-04-16,5000.0,2012-03-17 03:24:11,1.0,False,1,22,1.0,1.0,5000.00
1000011046,55,6,13,2015-08-29,19500.0,2015-07-04 08:35:03,1283.0,False,14,22,1283.0,1283.0,19500.00


## Make a new feature

In [3]:
data["duration"] = (data["deadline"]  - data['launched']).astype(int)

In [4]:
data.columns

Index(['category', 'main_category', 'currency', 'deadline', 'goal', 'launched',
       'pledged', 'state', 'backers', 'country', 'usd pledged',
       'usd_pledged_real', 'usd_goal_real', 'duration'],
      dtype='object')

In [5]:
feature = data[['category', 'main_category', 'currency', 'goal','duration',
       'pledged', 'backers', 'country', 'usd pledged',
       'usd_pledged_real', 'usd_goal_real']]
outcome = data[['state']]

## Load Necessary Library

In [6]:
from sklearn.metrics import accuracy_score,mean_absolute_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsClassifier,KNeighborsRegressor
from sklearn.pipeline import make_pipeline,make_union
from sklearn.preprocessing import MinMaxScaler, Imputer,PolynomialFeatures, StandardScaler
from sklearn.model_selection import GridSearchCV,KFold,train_test_split
import warnings
from sklearn.feature_selection import SelectPercentile,SelectKBest, VarianceThreshold
warnings.filterwarnings('ignore')

## Split test and train data set

In [7]:
train_features_small, validation_features, train_outcome_small, validation_outcome = train_test_split(
    feature,
    outcome,
    test_size = 0.2
)

In [10]:
pipe = make_pipeline(VarianceThreshold(0.1),
                     SelectKBest(),
                     KNeighborsClassifier(weights="distance")
                     )

params ={'kneighborsclassifier__n_neighbors': range(10,30,5),
         'selectkbest__k':range(5,11,2)
        }

grid_search_knn = GridSearchCV(pipe, param_grid = params,
                               cv = KFold(n_splits=10, shuffle=True),n_jobs = -1)

In [11]:
grid_search_knn.fit(train_features_small,train_outcome_small)

GridSearchCV(cv=KFold(n_splits=10, random_state=None, shuffle=True),
       error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('variancethreshold', VarianceThreshold(threshold=0.1)), ('selectkbest', SelectKBest(k=10, score_func=<function f_classif at 0x1272c8d08>)), ('kneighborsclassifier', KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=5, p=2,
           weights='distance'))]),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'kneighborsclassifier__n_neighbors': range(10, 30, 5), 'selectkbest__k': range(5, 11, 2)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [12]:
grid_search_knn.score(validation_features,validation_outcome)

0.8355925664799861

In [90]:
pred = grid_search_knn.predict(validation_features)

In [125]:
from sklearn.ensemble import RandomForestClassifier
pipe = make_pipeline(VarianceThreshold(0.1),
                     SelectKBest(k = 10),
                     RandomForestClassifier(max_features="auto",max_depth=4, min_samples_split=2)
                     )

params ={'randomforestclassifier__n_estimators':[10]
        }

grid_search_rf = GridSearchCV(pipe, param_grid = params,
                               cv = KFold(n_splits=10, shuffle=True),n_jobs = -1)

In [126]:
grid_search_rf.fit(train_features_small,train_outcome_small)

GridSearchCV(cv=KFold(n_splits=10, random_state=None, shuffle=True),
       error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('variancethreshold', VarianceThreshold(threshold=0.1)), ('selectkbest', SelectKBest(k=10, score_func=<function f_classif at 0x1272c8d08>)), ('randomforestclassifier', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=4, max_features='auto', max...obs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))]),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'randomforestclassifier__n_estimators': [10]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [127]:
grid_search_rf.score(validation_features,validation_outcome)

0.9615819509664374

In [162]:
temp = validation_outcome
temp["pred"] = pred
temp.state = temp.state.astype(int)

In [165]:
temp["diff"] = temp["state"] == temp["pred"]

In [172]:
len(temp[temp["diff"] == False])

2888

In [173]:
len(temp[temp["diff"] == True])

72285

In [116]:
from sklearn.neural_network import MLPClassifier

    
pipe = make_pipeline(VarianceThreshold(0.1),
                    SelectKBest(k = 10),
                     MLPClassifier(hidden_layer_sizes=(13,5,),  
                                  activation='relu', 
                                   batch_size='auto', 
                                  learning_rate_init=0.01, power_t=0.5, 
                                  max_iter=1000, shuffle=True,
                                  tol=0.0001, verbose=False, warm_start=False,
                                  momentum=0.9, nesterovs_momentum=True,
                                    early_stopping=False, validation_fraction=0.1,
                                  beta_1=0.9,beta_2=0.999, epsilon=1e-08)
                                )
params ={'mlpclassifier__alpha':[0.0001,0.001,0.01],
         'mlpclassifier__solver':['lbfgs',  'adam']
        }

grid_search_nn = GridSearchCV(pipe, param_grid = params,cv = KFold(n_splits=5, shuffle=True),n_jobs=-1)

In [111]:
train_outcome_small.state = train_outcome_small.state.astype(int)

In [117]:
grid_search_nn.fit(train_features_small,train_outcome_small)

GridSearchCV(cv=KFold(n_splits=5, random_state=None, shuffle=True),
       error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('variancethreshold', VarianceThreshold(threshold=0.1)), ('selectkbest', SelectKBest(k=10, score_func=<function f_classif at 0x1272c8d08>)), ('mlpclassifier', MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08...=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False))]),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'mlpclassifier__alpha': [0.0001, 0.001, 0.01], 'mlpclassifier__solver': ['lbfgs', 'adam']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [118]:
grid_search_nn.score(validation_features,validation_outcome)

0.6422518723477845